<a href="https://colab.research.google.com/github/Otavioalv/cursoLLM/blob/main/SuperVisionadoVSRegras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importações
import pandas as pd
from google.colab import files
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
files.upload()

In [4]:
Tweets = pd.read_csv("Tweets2.csv")
Tweets.shape

(74682, 4)

In [5]:
Tweets.head()

,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [6]:
Tweets.groupby(['sentiment']).size()

,0
sentiment,
Irrelevant,12990
Negative,22542
Neutral,18318
Positive,20832


In [7]:
Tweets.loc[Tweets['sentiment'] == 'Irrelevant', 'sentiment'] = 'Neutral'

Tweets.groupby(['sentiment']).size()

,0
sentiment,
Negative,22542
Neutral,31308
Positive,20832


In [8]:
Tweets = Tweets.dropna(subset=['text'])
Tweets.reset_index(drop=True, inplace=True)



In [9]:
Tweets.shape

(73996, 4)

In [10]:
# Supervisionado
# Pre processamento inicial com as tarefas supervisionadas

token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values)

In [11]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding='post', maxlen=100) # Remover padding e maxlen

In [12]:
labelencoder = LabelEncoder() # cria o objeto
y = labelencoder.fit_transform(Tweets['sentiment'])

print(y[0:10])

[2 2 2 2 2 2 2 2 2 2]


In [13]:
y =  to_categorical(y)
print(y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

X_test

array([[11,  7, 15, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [11,  7, 25, ...,  0,  0,  0],
       ...,
       [32, 39, 24, ...,  0,  0,  0],
       [17,  5,  4, ...,  0,  0,  0],
       [ 1, 54,  3, ...,  0,  0,  0]], dtype=int32)

In [15]:
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index), output_dim=128, input_length=X.shape[1]))

modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh',
                recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(modelo.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
modelo.fit(X_train, y_train, epochs=5, batch_size=500, verbose=True)

Epoch 1/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 159s 2s/step - accuracy: 0.4120 - loss: 1.0847
Epoch 2/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.4163 - loss: 1.0838
Epoch 3/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.4152 - loss: 1.0844
Epoch 4/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.4171 - loss: 1.0834
Epoch 5/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.4191 - loss: 1.0822


In [18]:
loss, accuracy = modelo.evaluate(X_test, y_test)

print(accuracy)

925/925 ━━━━━━━━━━━━━━━━━━━━ 68s 73ms/step - accuracy: 0.4242 - loss: 1.0804
0.42004796862602234


In [19]:
# VADER
mas = SentimentIntensityAnalyzer() # instancia o objeto
Tweets['vader_sentiment'] = ''

for y in range(len(Tweets.index)):
  x = mas.polarity_scores(Tweets['text'].iloc[y])

  del x['compound']

  maior = max(x, key=x.get)
  Tweets.loc[y, 'vader_sentiment'] = maior



In [22]:
Tweets.groupby(['vader_sentiment']).size()

,0
vader_sentiment,
neg,3660
neu,65581
pos,4755


In [24]:
Tweets.groupby(['sentiment']).size()

,0
sentiment,
Negative,22358
Neutral,30983
Positive,20655


In [26]:
Tweets.loc[Tweets['vader_sentiment'] == 'neu', 'vader_sentiment'] = 'Neutral'
Tweets.loc[Tweets['vader_sentiment'] == 'neg', 'vader_sentiment'] = 'Negative'
Tweets.loc[Tweets['vader_sentiment'] == 'pos', 'vader_sentiment'] = 'Positive'

Tweets.groupby(['vader_sentiment']).size()

,0
vader_sentiment,
Negative,3660
Neutral,65581
Positive,4755


In [27]:
y_pred = Tweets['vader_sentiment']
y_test = Tweets['sentiment']

cm = confusion_matrix(y_test, y_pred)

print(cm)

[[ 2004 19902   452]
 [ 1122 28384  1477]
 [  534 17295  2826]]


In [29]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.44886210065408944
